In [ ]:
#### LIBRARIES AND FUNCTIONS
using Plots
using JLD2
using Statistics
using Random
using StatsBase
using Integrals
using GenericSchur
using LaTeXStrings
using Pkg
using Combinatorics
using LinearAlgebra
using Base.Threads

function ArnoldMap_Integrate(xin,p)
    ν,σ,M,tr = p
    x0 , y0 = xin
    X = zeros(M+tr,2) # EDITED
    A = ones(2,2); A[1,1] = 2
    α = angle(ν) ; μ = abs(ν) 
    Xold = xin;
    for it in 1:M+tr
        sum = Xold[1] + Xold[2]
        num = μ*sin(2*π*sum - α)
        den = 1 - μ*cos(2*π*sum - α)

        map = mod.( A*Xold .+ 1/ π *atan(num/den) .+ σ*randn(2) , 1 )

        X[it,:] = map # EDITED
        Xold = map
    end
    return X[1+tr:end,:] # EDITED
end

function return_obs(X)
    x , y = X
    observables = zeros(5)
    observables[1] = sin(2*π*(x+y)) 
    observables[2] = cos(2*π*(x+y))
    observables[3] = sin(2*π*x)*cos(2*π*y)
    observables[4] = cos(2*π*x)*cos(2*π*y) 
    observables[5] = cos(2*π*x)*sin(6*π*y) 
    return observables
end

In [ ]:
M = 10^7; tr = 500 ; 
ν = 0.88*exp(1im*(0.7419+pi))
σ = 0.01

p = (ν,σ,Int(M),tr)
x0 = [0.3,0.2];

In [ ]:
# Unperturbed System
X = ArnoldMap_Integrate(x0,p);

avg = zeros(length(return_obs(X[1,:])),1)
for i in 1:size(X)[1]
    avg += return_obs(X[i,:])/size(X)[1]
end

In [ ]:
# Response Experiments
n_Traj = 6*10^6;
tr = 0; M = 50
p = (ν,σ,Int(M),tr)
amplitudes = [0.003,0.004,0.005,0.006]#,0.04]#,0.05][0.003,0.004,0.006]

In [ ]:
Data = Array{Matrix{Float64}}(undef, length(amplitudes))
g = zeros(M,size( return_obs(X[1,:]) )[1])

sum =  X[:,1] .+ X[:,2]
α = angle(ν) ; μ = abs(ν) 
pert_profile =  X[:,1]#sin.(2*π .* sum .- α) ./ ( 1 .+ μ^2  .- 2 .* μ .* cos.(2*π.*sum .- α) ) # sin.(2*π*X[:,2])#sin.(2*π.*(X[:,1] .- 2*X[:,2] ))#1/π .* sin.(2*π .* sum .- α) ./ ( 1 .+ μ^2  .- 2 .* μ .* cos.(2*π.*sum .- α) )
# log.(2 .+ cos.(2*π.*X[:,1] ))#
for (ii,ε) in enumerate(amplitudes)
    # ICs_plus = mod.( X.+ ε,1);
    # ICs_plus = copy(X)#mod.( X.+ ε,1);
    # ICs_plus[:,1] .= mod.( X[:,1] .+ ε *cos.(2*π*X[:,1]) , 1 )
    
    ICs_plus = mod.(X .+ ε*pert_profile,1)
    ICs_plus[:,2] .= X[:,2]
    G_plus = zeros(M,size( return_obs(X[1,:]) )[1])

    for i in 1:n_Traj
        Y = ArnoldMap_Integrate(ICs_plus[i,:],p);
        for i in 1:size(Y)[1]
            g[i,:] = return_obs(Y[i,:])
        end
        G_plus = G_plus .+ g ./n_Traj
    end
    ICs_plus = nothing; GC.gc()

    ICs_minus = mod.(X .- ε*pert_profile,1)
    ICs_minus[:,2] .= X[:,2]
    #ICs_minus = mod.( X.- ε,1);
    # ICs_minus = copy(X)
    # ICs_minus[:,1] .= mod.( ICs_minus[:,1] .- ε *cos.(2*π*X[:,1]) , 1 )
    G_minus = zeros(M,size( return_obs(X[1,:]) )[1])
    for i in 1:n_Traj
        Y = ArnoldMap_Integrate(ICs_minus[i,:],p);
        for i in 1:size(Y)[1]
            g[i,:] = return_obs(Y[i,:])
        end
        G_minus = G_minus .+ g ./n_Traj
    end
    ICs_minus = nothing; GC.gc()
    G = ( G_plus .- G_minus ) ./(2*ε);

    Data[ii] = G
end

In [ ]:
index = 5
plot(Data[1][:,index])
plot!(Data[2][:,index])
plot!(Data[3][:,index])
plot!(Data[3][:,index])

In [ ]:
jldsave("Data_GreenFunctions_01_X.jld2"; Data,p,n_Traj,amplitudes,return_obs)